<a href="https://colab.research.google.com/github/Senethlakshan/-hacktoberfest-2022/blob/main/data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Frist Quection - done

import pandas as pd
import numpy as np

# Load datasets
employee = pd.read_csv('/content/dataset/Employee.csv')
fields = pd.read_csv('/content/dataset/Fields.csv')
workcode = pd.read_csv('/content/dataset/WorkCode.csv')
workdetails = pd.read_csv('/content/dataset/WorkDetails.csv', low_memory=False)

# Convert 'DateofJoin' to datetime objects, coercing errors to NaT
employee['DateofJoin'] = pd.to_datetime(employee['DateofJoin'], errors='coerce')

# Function to print dataset properties
def print_dataset_properties(df, name):
    print(f"\n=== {name} Properties ===")
    print(f"Shape: {df.shape}")
    print("\nData Types:")
    print(df.dtypes)
    print("\nMissing Values:")
    print(df.isnull().sum())
    print("\nUnique Values per Column:")
    for col in df.columns:
        print(f"{col}: {df[col].nunique()} unique values")
    print("\nSample Statistics:")
    print(df.describe(include='all'))

# Analyze properties for each dataset
print_dataset_properties(employee, "Employee.csv")
print_dataset_properties(fields, "Fields.csv")
print_dataset_properties(workcode, "WorkCode.csv")
print_dataset_properties(workdetails, "WorkDetails.csv")

# Specific analyses for Employee.csv
print("\n=== Employee.csv Specifics ===")
print(f"Gender Distribution: \n{employee['Gender'].value_counts(normalize=True)}")
print(f"PAMACT Distribution: \n{employee['PAMACT'].value_counts(dropna=False, normalize=True)}")
print(f"DateofJoin Range: {employee['DateofJoin'].min()} to {employee['DateofJoin'].max()}")
print(f"EmpCode Uniqueness: {'Unique' if employee['EmpCode'].is_unique else 'Non-unique'}")

# Specific analyses for Fields.csv
print("\n=== Fields.csv Specifics ===")
print(f"Division Distribution: \n{fields['Division'].value_counts(normalize=True)}")
print(f"CropType Distribution: \n{fields['CropType'].value_counts(normalize=True)}")
print(f"Area (Ha) Summary: \n{fields['Area (Ha)'].describe()}")
print(f"ID Uniqueness: {'Unique' if fields['ID'].is_unique else 'Non-unique'}")

# Specific analyses for WorkCode.csv
print("\n=== WorkCode.csv Specifics ===")
print(f"WorCode Uniqueness: {'Unique' if workcode['WorCode'].is_unique else 'Non-unique'}")
print(f"Crop Distribution: \n{workcode['Crop'].value_counts(dropna=False, normalize=True)}")
print(f"Type Distribution: \n{workcode['Type'].value_counts(dropna=False, normalize=True)}")

# Specific analyses for WorkDetails.csv
print("\n=== WorkDetails.csv Specifics ===")
print(f"Year Range: {workdetails['Year'].min()} to {workdetails['Year'].max()}")
print(f"Work Distribution (Top 5): \n{workdetails['Work'].value_counts(normalize=True).head()}")
print(f"Workdayfraction Summary: \n{workdetails['Workdayfraction'].describe()}")
print(f"Qty Missing Percentage: {workdetails['Qty'].isnull().mean()*100:.2f}%")

# Demonstrate Relationships
print("\n=== Relationships Analysis ===")

# 1. Employee to Fields (1:1 via EmpCode=ID)
emp_fields = pd.merge(employee[['EmpCode', 'Gender']], fields[['ID', 'Division', 'CropType']],
                     left_on='EmpCode', right_on='ID', how='inner')
print(f"\nEmployee-Fields Merge (1:1): {emp_fields.shape[0]} rows (matches Employee/Fields row count)")
print(f"Sample Employee-Fields Merge:\n{emp_fields.head()}")

# 2. Employee to WorkDetails (1:M via EmpCode)
emp_workdetails = pd.merge(workdetails[['EmpCode', 'Year', 'Month', 'Work', 'Qty']],
                         employee[['EmpCode', 'Gender']],
                         on='EmpCode', how='inner')
print(f"\nEmployee-WorkDetails Merge (1:M): {emp_workdetails.shape[0]} rows")
print(f"Average records per EmpCode: {emp_workdetails.groupby('EmpCode').size().mean():.2f}")
print(f"Sample Employee-WorkDetails Merge:\n{emp_workdetails.head()}")

# 3. WorkDetails to WorkCode (M:1 via Work=WorCode)
workdetails_workcode = pd.merge(workdetails[['EmpCode', 'Work', 'Qty']],
                               workcode[['WorCode', 'Crop', 'Type']],
                               left_on='Work', right_on='WorCode', how='inner')
print(f"\nWorkDetails-WorkCode Merge (M:1): {workdetails_workcode.shape[0]} rows")
print(f"Sample WorkDetails-WorkCode Merge:\n{workdetails_workcode.head()}")

# 4. Full Merge (Employee, Fields, WorkDetails, WorkCode)
full_merge = pd.merge(pd.merge(pd.merge(workdetails,
                                      employee[['EmpCode', 'Gender', 'DateofJoin']],
                                      on='EmpCode', how='inner'),
                             fields[['ID', 'Division', 'CropType', 'Area (Ha)']],
                             left_on='EmpCode', right_on='ID', how='inner'),
                     workcode[['WorCode', 'Crop', 'Type']],
                     left_on='Work', right_on='WorCode', how='inner')
print(f"\nFull Merge Result: {full_merge.shape}")
print(f"Sample Full Merge:\n{full_merge.head()}")

# Verify Division from Fields.csv (authoritative)
print(f"\nDivision Source Check (Fields.csv):")
print(f"Unique Divisions in Fields: {fields['Division'].unique()}")
print(f"Division Distribution in Full Merge: \n{full_merge['Division'].value_counts(normalize=True)}")


=== Employee.csv Properties ===
Shape: (1932, 5)

Data Types:
Estate                object
EmpCode                int64
Gender                object
DateofJoin    datetime64[ns]
PAMACT                object
dtype: object

Missing Values:
Estate          0
EmpCode         0
Gender          0
DateofJoin    166
PAMACT          6
dtype: int64

Unique Values per Column:
Estate: 1 unique values
EmpCode: 1932 unique values
Gender: 2 unique values
DateofJoin: 713 unique values
PAMACT: 4 unique values

Sample Statistics:
       Estate       EmpCode Gender                     DateofJoin PAMACT
count    1932  1.932000e+03   1932                           1766   1926
unique      1           NaN      2                            NaN      4
top       EST           NaN      F                            NaN      A
freq     1932           NaN   1071                            NaN   1404
mean      NaN  7.305781e+05    NaN  1991-11-01 12:01:37.848244608    NaN
min       NaN  1.010000e+02    NaN         

In [21]:
import pandas as pd

# Load datasets from CSV files
employee = pd.read_csv('/content/dataset/Employee.csv')
fields = pd.read_csv('/content/dataset/Fields.csv')
workcode = pd.read_csv('/content/dataset/WorkCode.csv')
workdetails = pd.read_csv('/content/dataset/WorkDetails.csv', low_memory=False)  # Handle large file

# Verify loading by printing shapes
print("Dataset Shapes after Loading:")
print(f"Employee.csv: {employee.shape}")
print(f"Fields.csv: {fields.shape}")
print(f"WorkCode.csv: {workcode.shape}")
print(f"WorkDetails.csv: {workdetails.shape}")

# Optional: Display basic info for confirmation
print("\nBasic Info for Employee.csv:")
print(employee.info())
print("\nFirst few rows of Employee.csv:")
print(employee.head())

ParserError: Error tokenizing data. C error: Expected 7 fields in line 78912, saw 8


In [22]:
import pandas as pd

# Try reading up to the problematic line (add a small buffer for context)
try:
    problematic_data = pd.read_csv('/content/dataset/WorkDetails.csv', nrows=78912 + 10, low_memory=False)
    print("Successfully read up to line 78922")
    print(problematic_data.tail())  # Show last few rows
except pd.errors.ParserError as e:
    print(f"Error at: {e}")

Error at: Error tokenizing data. C error: Expected 7 fields in line 78912, saw 8



In [23]:
# Read the file as text and check line 78912 (lines are 1-indexed, so index 78911)
with open('/content/dataset/WorkDetails.csv', 'r') as f:
    lines = f.readlines()
    print("Line 78911 (0-indexed):", lines[78911].strip())  # The problematic line
    print("Line 78910 (previous):", lines[78910].strip() if len(lines) > 78910 else "N/A")
    print("Line 78912 (next):", lines[78912].strip() if len(lines) > 78912 else "N/A")

Line 78911 (0-indexed): 2014,2,6,527624,PLK,1,23,13
Line 78910 (previous): 2012,4,213716,PLK,1,27,9
Line 78912 (next): 2014,10,527624,PLK,1,23,13


In [24]:
# Read specific lines around the error
with open('/content/dataset/WorkDetails.csv', 'r') as f:
    lines = f.readlines()
    error_line_idx = 78911  # 0-indexed
    print(f"Header (line 0): {lines[0].strip()}")
    print(f"Line {error_line_idx} (0-indexed, line {error_line_idx+1} 1-indexed): {lines[error_line_idx].strip()}")
    print(f"Number of commas in error line: {lines[error_line_idx].count(',')}")
    print(f"Expected commas: 6 (for 7 fields)")

Header (line 0): ﻿Year,Month,EmpCode,Work,Workdayfraction,Qty,ExtraKilos
Line 78911 (0-indexed, line 78912 1-indexed): 2014,2,6,527624,PLK,1,23,13
Number of commas in error line: 7
Expected commas: 6 (for 7 fields)


In [27]:
import pandas as pd

# Step 1: Load Employee.csv, Fields.csv, and WorkCode.csv (no issues)
employee = pd.read_csv('/content/dataset/Employee.csv', encoding='utf-8-sig')
fields = pd.read_csv('/content/dataset/Fields.csv', encoding='utf-8-sig')
workcode = pd.read_csv('/content/dataset/WorkCode.csv', encoding='utf-8-sig')

# Step 2: Diagnose WorkDetails.csv issues (run once for inspection)
print("=== Diagnostic: Header and Problematic Line ===")
with open('/content/dataset/WorkDetails.csv', 'r', encoding='utf-8-sig') as f:
    lines = f.readlines()
    print(f"Header (BOM stripped): {lines[0].strip()}")
    error_line_idx = 78911  # 0-indexed
    print(f"Line {error_line_idx} (0-indexed): {lines[error_line_idx].strip()}")
    print(f"Number of commas: {lines[error_line_idx].count(',')} (expected 6)")

# Step 3: Load WorkDetails.csv with BOM handling and error skipping
try:
    # First try without skipping (after BOM fix)
    workdetails = pd.read_csv('/content/dataset/WorkDetails.csv',
                             encoding='utf-8-sig',
                             low_memory=False)
    print("Loaded WorkDetails without skipping (BOM handled).")
except pd.errors.ParserError as e:
    print(f"ParserError after BOM fix: {e}")
    # Fallback: Skip the bad line
    workdetails = pd.read_csv('/content/dataset/WorkDetails.csv',
                             encoding='utf-8-sig',
                             low_memory=False,
                             on_bad_lines='skip')
    print("Loaded with 1 bad line skipped.")

# Step 4: Verify all datasets
print("\n=== Dataset Shapes after Loading ===")
print(f"Employee.csv: {employee.shape}")
print(f"Fields.csv: {fields.shape}")
print(f"WorkCode.csv: {workcode.shape}")
print(f"WorkDetails.csv: {workdetails.shape} (note: 1 row skipped if malformed)")

print("\n=== Basic Info for WorkDetails.csv ===")
print(workdetails.info())

print("\n=== First Few Rows of WorkDetails.csv ===")
print(workdetails.head())

print("\n=== Validation: Sample for EmpCode=527624 (around problematic area) ===")
sample = workdetails[(workdetails['EmpCode'] == 527624) & (workdetails['Month'].isin([2, 10]))]
print(sample.head() if len(sample) > 0 else "Sample available (row fixed/skipped correctly).")

print(f"\n=== Missing Values in Key Columns ===")
print(workdetails[['Qty', 'ExtraKilos']].isnull().sum())

=== Diagnostic: Header and Problematic Line ===
Header (BOM stripped): Year,Month,EmpCode,Work,Workdayfraction,Qty,ExtraKilos
Line 78911 (0-indexed): 2014,2,6,527624,PLK,1,23,13
Number of commas: 7 (expected 6)
ParserError after BOM fix: Error tokenizing data. C error: Expected 7 fields in line 78912, saw 8

Loaded with 1 bad line skipped.

=== Dataset Shapes after Loading ===
Employee.csv: (1932, 5)
Fields.csv: (1932, 8)
WorkCode.csv: (483, 3)
WorkDetails.csv: (118241, 7) (note: 1 row skipped if malformed)

=== Basic Info for WorkDetails.csv ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118241 entries, 0 to 118240
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Year             118241 non-null  int64  
 1   Month            118241 non-null  int64  
 2   EmpCode          118240 non-null  object 
 3   Work             118240 non-null  object 
 4   Workdayfraction  118240 non-null  float64
 5   Qty

In [28]:
import pandas as pd

#Second Quection - done
# Load datasets with error handling for WorkDetails.csv
employee = pd.read_csv('/content/dataset/Employee.csv', encoding='utf-8-sig')
fields = pd.read_csv('/content/dataset/Fields.csv', encoding='utf-8-sig')
workcode = pd.read_csv('/content/dataset/WorkCode.csv', encoding='utf-8-sig')

# Robust load for WorkDetails.csv (handles BOM and malformed row)
try:
    workdetails = pd.read_csv('/content/dataset/WorkDetails.csv',
                             encoding='utf-8-sig',
                             low_memory=False)
except pd.errors.ParserError:
    workdetails = pd.read_csv('/content/dataset/WorkDetails.csv',
                             encoding='utf-8-sig',
                             low_memory=False,
                             on_bad_lines='skip')  # Skips 1 malformed row

# Final verification
print(f"Employee.csv: {employee.shape} - Columns: {employee.columns.tolist()}")
print(f"Fields.csv: {fields.shape} - Columns: {fields.columns.tolist()}")
print(f"WorkCode.csv: {workcode.shape} - Columns: {workcode.columns.tolist()}")
print(f"WorkDetails.csv: {workdetails.shape} - Columns: {workdetails.columns.tolist()}")
print(f"\nWorkDetails Sample (first 3 rows):")
print(workdetails.head(3))
print(f"\nData Types in WorkDetails:")
print(workdetails.dtypes)
print(f"\nMissing Values in WorkDetails (Qty, ExtraKilos): {workdetails[['Qty', 'ExtraKilos']].isnull().sum().to_dict()}")

Employee.csv: (1932, 5) - Columns: ['Estate', 'EmpCode', 'Gender', 'DateofJoin', 'PAMACT']
Fields.csv: (1932, 8) - Columns: ['ID', 'EState', 'Division', 'CropType', 'Field', 'Area (Ha)', 'NumberOfTree', 'Type']
WorkCode.csv: (483, 3) - Columns: ['WorCode', 'Crop', 'Type']
WorkDetails.csv: (118241, 7) - Columns: ['Year', 'Month', 'EmpCode', 'Work', 'Workdayfraction', 'Qty', 'ExtraKilos']

WorkDetails Sample (first 3 rows):
   Year  Month EmpCode Work  Workdayfraction   Qty  ExtraKilos
0  2012      6   41107  ABS              1.0  22.0         0.0
1  2012      7   41309  PLK              1.0  22.0         0.0
2  2012      9   41309  PLK              1.0  22.0         0.0

Data Types in WorkDetails:
Year                 int64
Month                int64
EmpCode             object
Work                object
Workdayfraction    float64
Qty                float64
ExtraKilos         float64
dtype: object

Missing Values in WorkDetails (Qty, ExtraKilos): {'Qty': 1, 'ExtraKilos': 2644}


In [32]:
#q 03

import pandas as pd
import numpy as np

# Load (from task b)
employee = pd.read_csv('/content/dataset/Employee.csv', encoding='utf-8-sig')

# Convert DateofJoin to datetime
employee['DateofJoin'] = pd.to_datetime(employee['DateofJoin'], format='%m/%d/%Y', errors='coerce')

# Check for outliers in DateofJoin
invalid_dates = employee[
    (employee['DateofJoin'].dt.year > 2015) |
    (employee['DateofJoin'].dt.year < 1900)
]
print(f"Employee: Invalid DateofJoin: {len(invalid_dates)} rows")

# Impute missing DateofJoin with median year (1990)
median_date = pd.to_datetime('1990-12-14')  # From task a
employee['DateofJoin'].fillna(median_date, inplace=True)

# Impute missing PAMACT with mode ('A')
employee['PAMACT'].fillna(employee['PAMACT'].mode()[0], inplace=True)

# Verify
print(f"Employee: Missing values post-cleaning: {employee.isnull().sum().to_dict()}")
print(f"Employee: PAMACT distribution:\n{employee['PAMACT'].value_counts(normalize=True)}")

Employee: Invalid DateofJoin: 0 rows
Employee: Missing values post-cleaning: {'Estate': 0, 'EmpCode': 0, 'Gender': 0, 'DateofJoin': 0, 'PAMACT': 0}
Employee: PAMACT distribution:
PAMACT
A    0.729814
T    0.258282
I    0.006211
P    0.005694
Name: proportion, dtype: float64


/tmp/ipython-input-3892788753.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  employee['PAMACT'].fillna(employee['PAMACT'].mode()[0], inplace=True)


In [36]:
#some worning there.I fixed it.

import pandas as pd
import numpy as np

# Load (from task b)
employee = pd.read_csv('/content/dataset/Employee.csv', encoding='utf-8-sig')

# Convert DateofJoin to datetime
employee['DateofJoin'] = pd.to_datetime(employee['DateofJoin'], format='%m/%d/%Y', errors='coerce')

# Check for outliers in DateofJoin
invalid_dates = employee[
    (employee['DateofJoin'].dt.year > 2015) |
    (employee['DateofJoin'].dt.year < 1900)
]
print(f"Employee: Invalid DateofJoin: {len(invalid_dates)} rows")

# Impute missing DateofJoin with median year (1990) - Fixed: Direct assignment
median_date = pd.to_datetime('1990-12-14')  # From task a
employee['DateofJoin'] = employee['DateofJoin'].fillna(median_date)  # No inplace

# Impute missing PAMACT with mode ('A') - Fixed: Direct assignment
mode_pamact = employee['PAMACT'].mode()[0]
employee['PAMACT'] = employee['PAMACT'].fillna(mode_pamact)  # No inplace

# Verify
print(f"Employee: Missing values post-cleaning: {employee.isnull().sum().to_dict()}")
print(f"Employee: PAMACT distribution:\n{employee['PAMACT'].value_counts(normalize=True)}")

Employee: Invalid DateofJoin: 0 rows
Employee: Missing values post-cleaning: {'Estate': 0, 'EmpCode': 0, 'Gender': 0, 'DateofJoin': 0, 'PAMACT': 0}
Employee: PAMACT distribution:
PAMACT
A    0.729814
T    0.258282
I    0.006211
P    0.005694
Name: proportion, dtype: float64


In [33]:
# Load
fields = pd.read_csv('/content/dataset/Fields.csv', encoding='utf-8-sig')

# Area (Ha) outlier detection
Q1_area = fields['Area (Ha)'].quantile(0.25)  # 0.8
Q3_area = fields['Area (Ha)'].quantile(0.75)  # 3.0
IQR_area = Q3_area - Q1_area
lower_area = Q1_area - 1.5 * IQR_area  # -2.5 (clipped at 0.3)
upper_area = Q3_area + 1.5 * IQR_area  # 6.3
outliers_area = fields[(fields['Area (Ha)'] < lower_area) | (fields['Area (Ha)'] > upper_area)]
print(f"Fields: Area (Ha) outliers: {len(outliers_area)} rows")

# Cap Area (Ha) outliers
fields['Area (Ha)'] = fields['Area (Ha)'].clip(lower=0.3, upper=upper_area)

# NumberOfTree: Replace 0 and cap outliers
zero_trees = fields[fields['NumberOfTree'] == 0]
print(f"Fields: Zero NumberOfTree: {len(zero_trees)} rows")
fields.loc[fields['NumberOfTree'] == 0, 'NumberOfTree'] = fields['NumberOfTree'].median()  # 17631

Q1_trees = fields['NumberOfTree'].quantile(0.25)  # 9257
Q3_trees = fields['NumberOfTree'].quantile(0.75)  # 29000
IQR_trees = Q3_trees - Q1_trees
upper_trees = Q3_trees + 1.5 * IQR_trees  # 58923.5
outliers_trees = fields[fields['NumberOfTree'] > upper_trees]
print(f"Fields: NumberOfTree outliers: {len(outliers_trees)} rows")
fields['NumberOfTree'] = fields['NumberOfTree'].clip(upper=upper_trees)

# Verify
print(f"Fields: Post-cleaning Area (Ha): {fields['Area (Ha)'].describe()}")
print(f"Fields: Post-cleaning NumberOfTree: {fields['NumberOfTree'].describe()}")

Fields: Area (Ha) outliers: 124 rows
Fields: Zero NumberOfTree: 21 rows
Fields: NumberOfTree outliers: 81 rows
Fields: Post-cleaning Area (Ha): count    1932.000000
mean        2.318478
std         1.956086
min         0.300000
25%         0.800000
50%         1.800000
75%         3.000000
max         6.300000
Name: Area (Ha), dtype: float64
Fields: Post-cleaning NumberOfTree: count     1932.000000
mean     21188.414079
std      14966.403948
min       2961.000000
25%       9302.000000
50%      17631.000000
75%      29000.000000
max      58547.000000
Name: NumberOfTree, dtype: float64


In [8]:
import pandas as pd
import numpy as np

# Load WorkCode.csv
workcode = pd.read_csv('/content/dataset/WorkCode.csv', encoding='utf-8-sig')

# Identify duplicates in WorCode
duplicates = workcode[workcode['WorCode'].duplicated(keep=False)]  # Show all instances
print(f"WorkCode: Duplicate WorCode values:\n{duplicates[['WorCode', 'Crop', 'Type']]}")
duplicate_count = len(workcode[workcode['WorCode'].duplicated()])
print(f"WorkCode: Duplicate WorCode rows: {duplicate_count}")

# Remove duplicates, keeping first occurrence
workcode = workcode.drop_duplicates(subset='WorCode', keep='first')

# Impute missing Type with mode ('R') - Fixed: Direct assignment
mode_type = workcode['Type'].mode()[0]
workcode['Type'] = workcode['Type'].fillna(mode_type)  # No inplace

# Verify cleaning
print(f"WorkCode: Shape post-cleaning: {workcode.shape}")
print(f"WorkCode: Missing Type: {workcode['Type'].isnull().sum()}")
print(f"WorkCode: WorCode uniqueness: {workcode['WorCode'].is_unique}")
print(f"WorkCode: Type distribution:\n{workcode['Type'].value_counts(normalize=True)}")

WorkCode: Duplicate WorCode values:
    WorCode Crop Type
33      CT3    T    R
108     FSU    T    R
111     FSU    T    R
182     CT3    T    R
WorkCode: Duplicate WorCode rows: 2
WorkCode: Shape post-cleaning: (481, 3)
WorkCode: Missing Type: 0
WorkCode: WorCode uniqueness: True
WorkCode: Type distribution:
Type
R    0.702703
O    0.162162
C    0.135135
Name: proportion, dtype: float64


In [35]:
# Load (post-task b fix)
workdetails = pd.read_csv('/content/dataset/WorkDetails.csv',
                         encoding='utf-8-sig',
                         low_memory=False,
                         on_bad_lines='skip')

# Workdayfraction: Clip to plausible range
workdetails['Workdayfraction'] = workdetails['Workdayfraction'].clip(lower=0.5, upper=2.0)
outliers_wdf = workdetails[(workdetails['Workdayfraction'] < 0.5) | (workdetails['Workdayfraction'] > 2.0)]
print(f"WorkDetails: Workdayfraction outliers: {len(outliers_wdf)}")

# Qty: Investigate uniform 22 and outliers (exclude ABS)
non_abs = workdetails[workdetails['Work'] != 'ABS']
qty_22 = len(non_abs[non_abs['Qty'] == 22])
print(f"WorkDetails: Non-ABS rows with Qty=22: {qty_22} ({qty_22/len(non_abs)*100:.2f}%)")

# Replace Qty=22 with median of non-ABS, non-22
non_abs_non_22_median = non_abs[non_abs['Qty'] != 22]['Qty'].median()  # ~52 from task a
workdetails.loc[(workdetails['Work'] != 'ABS') & (workdetails['Qty'] == 22), 'Qty'] = non_abs_non_22_median

# Cap Qty outliers (non-ABS)
Q1_qty = non_abs['Qty'].quantile(0.25)  # ~34
Q3_qty = non_abs['Qty'].quantile(0.75)  # ~71
IQR_qty = Q3_qty - Q1_qty
upper_qty = Q3_qty + 1.5 * IQR_qty  # ~126.5
outliers_qty = non_abs[non_abs['Qty'] > upper_qty]
print(f"WorkDetails: Qty outliers (non-ABS): {len(outliers_qty)}")
workdetails.loc[workdetails['Work'] != 'ABS', 'Qty'] = workdetails.loc[workdetails['Work'] != 'ABS', 'Qty'].clip(upper=upper_qty)

# ExtraKilos: Cap outliers
Q1_extra = workdetails['ExtraKilos'].quantile(0.25)  # 0
Q3_extra = workdetails['ExtraKilos'].quantile(0.75)  # 9
IQR_extra = Q3_extra - Q1_extra
upper_extra = Q3_extra + 1.5 * IQR_extra  # 13.5
outliers_extra = workdetails[workdetails['ExtraKilos'] > upper_extra]
print(f"WorkDetails: ExtraKilos outliers: {len(outliers_extra)}")
workdetails['ExtraKilos'] = workdetails['ExtraKilos'].clip(upper=upper_extra)

# Verify
print(f"WorkDetails: Post-cleaning Qty (non-ABS):\n{workdetails[workdetails['Work'] != 'ABS']['Qty'].describe()}")
print(f"WorkDetails: Post-cleaning ExtraKilos:\n{workdetails['ExtraKilos'].describe()}")

WorkDetails: Workdayfraction outliers: 0
WorkDetails: Non-ABS rows with Qty=22: 18930 (18.28%)
WorkDetails: Qty outliers (non-ABS): 0
WorkDetails: ExtraKilos outliers: 79
WorkDetails: Post-cleaning Qty (non-ABS):
count    103539.000000
mean         24.790958
std           1.213927
min           6.000000
25%          24.000000
50%          25.000000
75%          26.000000
max          27.000000
Name: Qty, dtype: float64
WorkDetails: Post-cleaning ExtraKilos:
count    115597.000000
mean          5.416131
std           6.043476
min           0.000000
25%           0.000000
50%           3.000000
75%          10.000000
max          25.000000
Name: ExtraKilos, dtype: float64


In [2]:
#Q 04

import pandas as pd
import numpy as np

# Load cleaned WorkDetails.csv (from task c)
workdetails = pd.read_csv('/content/dataset/WorkDetails.csv',
                         encoding='utf-8-sig',
                         low_memory=False,
                         on_bad_lines='skip')

# Apply task c cleaning (repeated for completeness)
workdetails['Workdayfraction'] = workdetails['Workdayfraction'].clip(lower=0.5, upper=2.0)
non_abs = workdetails[workdetails['Work'] != 'ABS']
non_abs_non_22_median = non_abs[non_abs['Qty'] != 22]['Qty'].median()
workdetails.loc[(workdetails['Work'] != 'ABS') & (workdetails['Qty'] == 22), 'Qty'] = non_abs_non_22_median
Q1_qty = non_abs['Qty'].quantile(0.25)
Q3_qty = non_abs['Qty'].quantile(0.75)
IQR_qty = Q3_qty - Q1_qty
upper_qty = Q3_qty + 1.5 * IQR_qty
workdetails.loc[workdetails['Work'] != 'ABS', 'Qty'] = workdetails.loc[workdetails['Work'] != 'ABS', 'Qty'].clip(upper=upper_qty)
Q1_extra = workdetails['ExtraKilos'].quantile(0.25)
Q3_extra = workdetails['ExtraKilos'].quantile(0.75)
IQR_extra = Q3_extra - Q1_extra
upper_extra = Q3_extra + 1.5 * IQR_extra
workdetails['ExtraKilos'] = workdetails['ExtraKilos'].clip(upper=upper_extra)

# Pre-imputation analysis
print("=== Pre-Imputation Missing Values ===")
missing_qty = workdetails['Qty'].isnull().sum()
missing_extra = workdetails['ExtraKilos'].isnull().sum()
print(f"Qty missing: {missing_qty} ({missing_qty/len(workdetails)*100:.2f}%)")
print(f"ExtraKilos missing: {missing_extra} ({missing_extra/len(workdetails)*100:.2f}%)")
abs_missing_qty = workdetails[workdetails['Work'] == 'ABS']['Qty'].isnull().sum()
print(f"Qty missing in ABS rows: {abs_missing_qty} ({abs_missing_qty/len(workdetails[workdetails['Work'] == 'ABS'])*100:.2f}% of ABS rows)")

# Apply constant value imputation (0)
workdetails['Qty'] = workdetails['Qty'].fillna(0)
workdetails['ExtraKilos'] = workdetails['ExtraKilos'].fillna(0)

# Post-imputation validation
print("\n=== Post-Imputation Missing Values ===")
print(f"Qty missing: {workdetails['Qty'].isnull().sum()}")
print(f"ExtraKilos missing: {workdetails['ExtraKilos'].isnull().sum()}")
print("\n=== Post-Imputation Qty Stats (non-ABS) ===")
print(workdetails[workdetails['Work'] != 'ABS']['Qty'].describe())
print("\n=== Post-Imputation ExtraKilos Stats ===")
print(workdetails['ExtraKilos'].describe())
print("\n=== Sample ABS Rows Post-Imputation ===")
print(workdetails[workdetails['Work'] == 'ABS'][['Year', 'Month', 'EmpCode', 'Work', 'Qty', 'ExtraKilos']].head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset/WorkDetails.csv'